In [2]:
dislpay = """

#/////////////////////////////////////////////////////////////////////////////////#
#    Code for compute Jarzynski's equality                                        #
#     reference paper:                                                            #
#        Free energy calculation of single molecular interaction using            #
#        Jarzynski's identity method: the case of HVI-1 protease inhibitorsystem  #
#         Acta Mech. Sin. (2012) 28(3):891–903. DOI 10.1007/s10409-012-0112-9     #
#                                                                                 #
#        compute Work and PMF using:                                              #
#            ΔW = (Fi+F0)*(Zi-Z0),                                                #
#                        (Fi-F0) are force applied to displazament                #
#                        (Zi-Z0) are displazamento, all measure into pN/A         #
#                                v= velocity pulling, dt = step integration       #
#                            convert W pN/A to kcal/mol, using:                   #
#                                    1 kcal/mol/A = ~69.479 pN/A^2                #
#            PMF = <W> -1/2βσ^2,                                                  #
#                        where σ^2 is:  σ^2 = <W^2>-<W>^2.                        #
#                                                                                 #
#     @autor: Ropón-Palacios G, PhD. E-mail: groponp@gamil.com                    #
#     date: June 14, 2021.                                                        #                                          
#/////////////////////////////////////////////////////////////////////////////////#
"""

##; Import libries 
import pandas as pd
import numpy as np
from __future__ import division



##; Here set the necesary constants so that they can be used futher down. 

v  = 0.0001          ##; velocity pulling 
dt = 0.002           ##; integration step 
T  = 300             ##, temperature in K 
kb = 0.001982923700  ##; boltzman constat intk kcal/mol K  
beta = 1/(kb*T)      ##; beta value 
row, col = 20001,10       ##; 7000 is datapoint, 20 number trajs smd. 

fname = "force_matrix.dat"

#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"""
    Function I:
        In this part of the script, we begin by reading an array of NxM dimensions,
        a table in case cannot be understood, wich has N values of rows and M values
        of columns, in which M correspond to the number of sampled trajectories, and 
        N to number of work data saved for each trajectory. 

        Note: Work is already previously calculated in kcal/mol using a TclForce script
        during smd simualtion.
    """
    
df = pd.read_csv(fname, sep="\t")
matrix_f = df.to_numpy()    ##; create an numpy matrix for Work
print(matrix_f)

[[-1.682913 -5.934932 -0.90483  ... -1.496731 -2.347691 -1.965275]
 [ 0.020768 -5.722008  0.262114 ... -0.079935 -2.640416  0.727232]
 [ 1.011459 -3.3936    2.275577 ... -1.422112 -4.605937  0.741469]
 ...
 [-0.674924  2.890915 -0.100681 ...  2.071568  2.792674  5.172366]
 [ 0.766946  2.719628  1.283833 ...  0.781235  1.43747   4.013109]
 [ 2.647283  1.045314  1.614493 ...  3.215194  1.039033  0.794843]]


In [9]:
###; calculate work 
dt = 0.1
v = 0.01 
tmpwork = matrix_f * v * dt 
cum_work = np.cumsum(tmpwork, axis=0, dtype=float)
cum_work

df2 = pd.DataFrame(cum_work)
df2

####; exponent of work 




0          1          2          3          4          5  \
0      -0.001683  -0.005935  -0.000905  -0.000839  -0.002374  -0.002105   
1      -0.001662  -0.011657  -0.000643  -0.001362  -0.006874  -0.005023   
2      -0.000651  -0.015051   0.001633  -0.004145  -0.008899  -0.007445   
3      -0.001455  -0.019088   0.002803  -0.007232  -0.009360  -0.010528   
4      -0.003990  -0.021664   0.005101  -0.007886  -0.010842  -0.013263   
...          ...        ...        ...        ...        ...        ...   
19995  22.877146  21.773968  23.688434  26.851092  21.591546  24.898573   
19996  22.876848  21.777452  23.685843  26.848639  21.590604  24.897575   
19997  22.876173  21.780343  23.685742  26.848373  21.590640  24.894720   
19998  22.876940  21.783063  23.687026  26.849251  21.590899  24.897803   
19999  22.879588  21.784108  23.688640  26.847932  21.592853  24.900990   

               6          7          8          9  
0      -0.003333  -0.001497  -0.002348  -0.001965  
1      -0.004889  -0.001577  -0.004988  -0.001238  
2      -0.006944  -0.002999  -0.009594  -0.000497  
3      -0.007164  -0.006637  -0.010129  -0.002467  
4      -0.009042  -0.006976  -0.013702  -0.005736  
...          ...        ...        ...        ...  
19995  22.088812  24.296839  27.056964  23.780205  
19996  22.088638  24.298086  27.057918  23.781277  
19997  22.089635  24.300157  27.060711  23.786450  
19998  22.088662  24.300939  27.062148  23.790463  
19999  22.088956  24.304154  27.063187  23.791258  

[20000 rows x 10 columns]

In [ ]:
#++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"""
    Function II: 
        1. In this function we proceed to perform the calculation of the average of the work squared <w^2>
            with which first new NxM matrix is generated by squaring each value of work and then the squared
            values are averaged for each row. 
            
        2. Next, the square of the average of the work <w>^2 is calculated, for wich the initial matrix is
            calcuated, for which the initial matrix is used the rows are sum to obtain the average and each
            average value is squared. 
            
        3. Finally from the original matrix we calculate only the average of the work <w> per rows. 
            
        Both the <w^2>, <w>^2 y <w> are stired in 1D array. 
            alias: 
                <w^2> = w1
                <w>^2 = w2
                <w>   = w0
"""
###; calcuate <w^2> 
matrix_w1_square  = np.zeros((row,col))
matrix_w1_square  = np.square(matrix_w) 
matrix_w1_average = [] 
matrix_w1_average = matrix_w1_square.mean(axis=1)
 
###; calculate <w>^2
matrix_w2_average = []
matrix_w2_average = matrix_w.mean(axis=1)
matrix_w2_average_squared = np.square(matrix_w2_average)

###; calculate <w> 
matrix_w0_average = []
matrix_w0_average = matrix_w.mean(axis=1)

#print(matrix_w1_average)
#print(matrix_w2_average_squared)
print(matrix_w0_average)


In [ ]:
#+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
"""
    Function III: 
        In this last function we are going to reconstruct the PMF, through the iteration of the work
        values calculated before. 
        Equation: 
                PMF = <W> -1/2βσ^2,                                                  
                     where σ^2 is:  σ^2 = <W^2>-<W>^2.    
"""
diff_w1_w2 = []
zip_object = zip(matrix_w1_average, matrix_w2_average_squared)

for i,j in zip_object: 
    diff_w1_w2.append(i-j)
    #print(diff_w1_w2)

pmf = [] 

for i in range(len(matrix_w0_average)): 
    deltaG = matrix_w0_average[i]*(-0.5*beta*diff_w1_w2[i])
    pmf.append(deltaG)
    #print(deltaG)
print(pmf)


In [ ]:
#++++++++++++++++++++++++++++++++++++++++
"""
    Function IV: 
        Write PMF to an file .dat 
"""
ofile = "PMF_reconstruct_kcal_mol.dat"
with open(ofile, 'w') as f:
    for item in pmf:
        f.write("%s\n" % item)

print("Check out your pmf into this directory: ", ofile)

In [ ]:
!vi PMF_reconstruct_kcal_mol.dat

In [ ]:
f1 =  2.647283   
w1 = f1*0.01*0.1
T = 0.6 
texp = np.exp(-w1/T)
fexp = -T*np.log(texp/10)
print(w1,texp,fexp)


In [ ]:
f1 = -1.454398   
fsum = 0
fsum = fsum + f1 * 0.1 * 0.01 
fsum = fsum +  -1.682913  * 0.1 * 0.01
fsum = fsum + 0.020768 * 0.1 * 0.01


print(fsum)
